In [2]:
import numpy as np
from scipy.stats import norm


In [3]:
N = norm.cdf

def BS_CALL(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r*T)* N(d2)

def BS_PUT(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma* np.sqrt(T)
    return K*np.exp(-r*T)*N(-d2) - S*N(-d1)

# FDM

In [ ]:
#All inputs

S_star = 500    #Stock price
T = 1           #Expiry date
E = 100         #Strike price or Exercise price
N = 50          #Price Levels
M = 365         #Time steps
D = 0.00        #Dividend
r = 0.05        #risk free rate
sigma = 0.2     #volatility

ds = S_star/N
dt = T/M

#Payoff function
def payoff(price,exercise):
    return max(price - exercise,0) #Call option: return max(price - exercise,0) ||| Put option: max(exercise - price,0)

#Check Conditional Stability for Explicit Scheme
if dt <= 1/(sigma*N)**2:
    print("Pass")
else:
    print("Recheck Conditional Stability")

In [ ]:
# European Explicit %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
V = np.zeros((N+1,M+1))

#Final Payoff Condition (0<=n<=N , t=T)
for n in range(N+1):
    V[n][M] = payoff(n*ds,E)


#Boundary Condition (S=0)
b_0 = 1-(r*dt)                 #Call option & Put Option 

for m in range(M,0,-1):
    V[0][m-1] = V[0][m]*b_0

#Differential Equation
a_N = 0.5*((N**2)*(sigma**2)-N*(r-D))*dt
b_N = 1-(r+(N**2)*(sigma**2))*dt
c_N = 0.5*((N**2)*(sigma**2)+N*(r-D))*dt

for m in range(M,0,-1):
    
    for n in range(1,N,1):
        a_n = 0.5*((n**2)*(sigma**2)-n*(r-D))*dt
        b_n = 1-(r+(n**2)*(sigma**2))*dt
        c_n = 0.5*((n**2)*(sigma**2)+n*(r-D))*dt
        
        V[n,m-1] = a_n*V[n-1,m] + b_n*V[n,m] + c_n*V[n+1,m]
    #Boundary Condition (S=S_star)
    V[N,m-1] = (a_N - c_N)*V[N-1,m] + (b_N + 2*c_N)*V[N,m]     #Call option & Put Option(For put options this is close to 0)

#np.set_printoptions(precision=6)
print(V)
V[10][0]

# American Explicit %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
VA = np.zeros((N+1,M+1))

#Final Payoff Condition (0<=n<=N , t=T)%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
for n in range(N+1):
    VA[n][M] = payoff(n*ds,E)

#Boundary Condition (S=0)%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
b_0 = 1-(r*dt)
for m in range(M,0,-1): 
    if VA[0][m]*b_0 < VA[n][M]:        #payoff check (American Option)
        VA[N][m-1] = VA[n][M]
    else:
        VA[N][m-1] = VA[0][m]*b_0    

    
#Differential Equation%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
a_N = 0.5*((N**2)*(sigma**2)-N*(r-D))*dt
b_N = 1-(r+(N**2)*(sigma**2))*dt
c_N = 0.5*((N**2)*(sigma**2)+N*(r-D))*dt

for m in range(M,0,-1):    
    for n in range(1,N,1):
        a_n = 0.5*((n**2)*(sigma**2)-n*(r-D))*dt
        b_n = 1-(r+(n**2)*(sigma**2))*dt
        c_n = 0.5*((n**2)*(sigma**2)+n*(r-D))*dt
        
        result = a_n*VA[n-1][m] + b_n*VA[n][m] + c_n*VA[n+1][m] 

        if result < payoff(n*ds,E):       #payoff check (American Option)
            VA[n][m-1] = payoff(n*ds,E)
        else:
            VA[n][m-1] = result
    #Boundary Condition (S=S_star)%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    result = (a_N-c_N)*VA[N-1][m] + (b_N+2*c_N)*VA[N][m]

    if result < payoff(N*ds,E):
        VA[N][m-1] = payoff(N*ds,E)
    else:
        VA[N][m-1] = result    

print(np.around(VA,6))

In [ ]:
# European Explicit %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
VI = np.zeros((N+1,M+1))

#Final Payoff Condition (0<=n<=N , t=T)%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
for n in range(N+1):
    VI[n][M] = payoff(n*ds,E)

#Boundary Condition (S = S_star)%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
ai_N = N*(r-D)*dt
bi_N = 1-(N*(r-D)-r)*dt
#---------------------------------
ai_n = np.empty(N)
bi_n = np.empty(N)
ci_n = np.empty(N)

#setup abc elements
for n in range(1,N,1):
    ai_n[n] = -0.5*((sigma**2)*((n)**2)-(n)*(r-D))*dt
ai_n = np.delete(ai_n,0)
for n in range(0,N,1): 
    bi_n[n] = 1+((sigma**2)*(n**2)+r)*dt
    ci_n[n] = -0.5*((sigma**2)*(n**2)+n*(r-D))*dt

#create an abc metrix
abc = np.diag(np.append(bi_n,0)) + np.diag(ci_n,1) + np.diag(np.append(ai_n,0),-1)
abc[N][N] = bi_N
abc[N][N-1] = ai_N

#solve the matrix system%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
for m in range(M,0,-1):
     VI[:,m-1] = np.linalg.solve(abc,VI[:,m])

print(np.around(VI,10))
VI[10][0]


# American Explicit %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
VIA = np.zeros((N+1,M+1))
#Final Payoff Condition (0<=n<=N , t=T)%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
for n in range(N+1):
    VIA[n][M] = payoff(n*ds,E)
    
#Boundary Condition (S = S_star)%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
ai_N = N*(r-D)*dt
bi_N = 1-(N*(r-D)-r)*dt
#---------------------------------
ai_n = np.empty(N)
bi_n = np.empty(N)
ci_n = np.empty(N)

#setup abc elements
for n in range(1,N,1):
    ai_n[n] = -0.5*((sigma**2)*((n)**2)-(n)*(r-D))*dt
ai_n = np.delete(ai_n,0)
for n in range(0,N,1): 
    bi_n[n] = 1+((sigma**2)*(n**2)+r)*dt
    ci_n[n] = -0.5*((sigma**2)*(n**2)+n*(r-D))*dt

#create an abc metrix
abc = np.diag(np.append(bi_n,0)) + np.diag(ci_n,1) + np.diag(np.append(ai_n,0),-1)
abc[N][N] = bi_N
abc[N][N-1] = ai_N

#solve the matrix system%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
for m in range(M,0,-1):
    VIA[:,m-1] = np.linalg.solve(abc,VIA[:,m])
    for n in range(0,N+1,1):       #payoff check (American Option)
        if VIA[n,m] < payoff(n*ds,E):
            VIA[n,m] = payoff(n*ds,E)            
for n in range(0,N+1,1):           #payoff check (American Option) at m = 0  
    if VIA[n,0] < payoff(n*ds,E):
        VIA[n,0] = payoff(n*ds,E)

print(np.around(VIA,6))

In [ ]:
# The Theta Method %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
VT = np.zeros((N+1,M+1))
VTABC = np.zeros((N+1,M+1))

theta = 0   # 1 = implicit , 0 = explicit

#Final Payoff Condition (0<=n<=N , t=T)%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
for n in range(N+1):
    VT[n][M] = payoff(n*ds,E)
    VTABC[n][M] = payoff(n*ds,E)
    
#Boundary Condition (S=0)
b_0 = 1-(r*dt)

for m in range(M,0,-1):
    VTABC[0][m-1] = VTABC[0][m]*b_0

#Boundary Condition (S = S_star)%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
at_N = theta*N*(r-D)*dt
bt_N = 1-theta*(N*(r-D)-r)*dt

At_N = -(1-theta)*N*(r-D)*dt
Bt_N = 1+(1-theta)*(N*(r-D)-r)*dt
#---------------------------------
at_n = np.empty(N)
bt_n = np.empty(N)
ct_n = np.empty(N)

At_n = np.empty(N)
Bt_n = np.empty(N)
Ct_n = np.empty(N)

#setup ABCt elements
for n in range(1,N,1):
    At_n[n] = 0.5*(1-theta)*((sigma**2)*((n)**2)-(n)*(r-D))*dt
At_n = np.delete(At_n,0)

for n in range(0,N,1): 
    Bt_n[n] = 1-(1-theta)*((sigma**2)*(n**2)+r)*dt
    Ct_n[n] = 0.5*(1-theta)*((sigma**2)*(n**2)+n*(r-D))*dt

#create an abc metrix
VTABC = np.diag(np.append(Bt_n,0)) + np.diag(Ct_n,1) + np.diag(np.append(At_n,0),-1)
VTABC[N][N] = Bt_N
VTABC[N][N-1] = At_N


#setup abct elements
for n in range(1,N,1):
    at_n[n] = -0.5*theta*((sigma**2)*((n)**2)-(n)*(r-D))*dt
at_n = np.delete(at_n,0)

for n in range(0,N,1): 
    bt_n[n] = 1+theta*((sigma**2)*(n**2)+r)*dt
    ct_n[n] = -0.5*theta*((sigma**2)*(n**2)+n*(r-D))*dt

#create an abc metrix
abct = np.diag(np.append(bt_n,0)) + np.diag(ct_n,1) + np.diag(np.append(at_n,0),-1)
abct[N][N] = bt_N
abct[N][N-1] = at_N


#solve the matrix system%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
for m in range(M,0,-1):
     VT[:,m-1] = np.linalg.solve(np.linalg.inv(VTABC) @ abct,VT[:,m])  ##### For American Options change here %%%%% 
        
# print(VTABC)
# print(abct)
print(np.around(VT,10))
VT[10][0]